## 13. 아이유팬이 좋아할 만한 다른 아티스트 찾기

##### 0. 라이브러리 로드  

In [141]:
import numpy as np
import pandas as pd
import scipy
import implicit
import os

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.22.4
1.8.1
0.6.1


##### 1) 데이터 준비와 전처리

In [142]:
# rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
rating_base_path='/home/evergrin/python/notebook/data/recommenddata_iu/'
file_name = 'ratings.dat'
rating_file_path = os.path.join(rating_base_path, file_name)

ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [143]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [144]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [145]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [146]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
# movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
movie_file_path = os.path.join(rating_base_path, 'movies.dat')
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [147]:
# 검색을 쉽게 하기 위해 문자열을 소문자로 바꿔줍시다.
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()

print(movies.info())
movies.head(10)

movies['title'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB
None


array(['toy story (1995)', 'jumanji (1995)', 'grumpier old men (1995)',
       ..., 'tigerland (2000)', 'two family house (2000)',
       'contender, the (2000)'], dtype=object)

##### 2) 분석해 봅시다.

In [148]:
ratings.head()

# 영화 개수
movie_count = len(movies['movie_id'])

# 사용자수
user_count = ratings['user_id'].nunique()

print("movie count:{}, user_count:{}".format(movie_count, user_count))

# 인기는 평점 개수로함.
ratings_count = ratings.groupby('movie_id')['counts'].count()
ratings_count.sort_values(ascending=False).head(30)

movie count:3883, user_count:6039


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

##### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.  

In [149]:
# title_unique = movies['title'].unique()
title_to_mov_id = {k:v for k,v in zip(list(movies['title']), list(movies['movie_id']))}

# # 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['old yeller (1957)' , 'fanny and alexander (1982)' ,'verdict, the (1982)' ,'contender, the (2000)' ,'sabrina (1995)']

my_favorite = [title_to_mov_id[title] for title in my_favorite]

next_user_id = ratings['user_id'].max() + 1
my_playlist = pd.DataFrame({'user_id': [next_user_id]*5, 'movie_id': my_favorite, 'counts':[4]*5, 'timestamp': [978310760, 978320760, 978330760, 978340760, 978350760]})

if not ratings.isin({'user_id':[next_user_id]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

# ratings = ratings.reset_index()
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

/tmp/ipykernel_39852/177617358.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다.


,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,6041,1012,4,978310760
1,6041,2068,4,978320760
2,6041,3068,4,978330760
3,6041,3952,4,978340760
4,6041,7,4,978350760


##### 4) CSR matrix를 직접 만들어 봅시다.

In [150]:
from scipy.sparse import csr_matrix

# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = movies['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[next_user_id]) 
print(title_to_mov_id['contender, the (2000)'])

# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_rating = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_rating) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_rating   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')


temp_movie_rating = ratings['movie_id'].map(movie_to_idx.get).dropna()
# temp_movie_rating = ratings['movie_id'].map(movie_to_idx.get)
if len(temp_movie_rating) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('rating movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_rating   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('rating movie_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movies['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movies):
    print('movie movie_id column indexing OK!!')
    movies['movie_id'] = temp_movie_data
else:
    print('movie movie_id column indexing Fail!!')


num_user = ratings['user_id'].nunique()
num_movie = movies['movie_id'].nunique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

csr_ratings = csr_matrix((ratings.counts, (ratings.movie_id, ratings.user_id)), shape=(num_movie, num_user))


6039
3952
user_id column indexing OK!!
rating movie_id column indexing OK!!
movie movie_id column indexing OK!!


In [152]:
num_movie

3883

In [153]:
print(user_to_idx[next_user_id])

6039


##### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [154]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

# 모델 훈련
als_model.fit(csr_ratings_transpose)


/home/evergrin/anaconda3/envs/pyenv_3912/lib/python3.9/site-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.3791463375091553 seconds
  warnings.warn(
100%|██████████| 15/15 [01:13<00:00,  4.90s/it]


In [155]:
len(als_model.item_factors)

3883

##### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [156]:
uid, mov_id = user_to_idx[next_user_id], movie_to_idx[title_to_mov_id['grumpier old men (1995)']]
uid_vector, mov_id_vector = als_model.user_factors[uid], als_model.item_factors[mov_id]

print(np.dot(uid_vector, mov_id_vector))

uid, mov_id = user_to_idx[next_user_id], movie_to_idx[title_to_mov_id['two family house (2000)']]
uid_vector, mov_id_vector = als_model.user_factors[uid], als_model.item_factors[mov_id]

print(np.dot(uid_vector, mov_id_vector))


0.011901446
0.036943316


##### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [157]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
mov_id_to_title = {v:k for k,v in title_to_mov_id.items()}

def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[title_to_mov_id[movie_name.lower()]]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [mov_id_to_title[idx_to_movie[i[0]]] for i in similar_movie]
    return similar_movie

get_similar_movie('Waiting to Exhale (1995)')

['waiting to exhale (1995)', 'jumanji (1995)']

In [185]:
user = user_to_idx[next_user_id]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=False)

movie_recommended = [mov_id_to_title[idx_to_movie[i]] for i in movie_recommended[0]]

print(movie_recommended)


['contender, the (2000)', 'almost famous (2000)', 'nurse betty (2000)', 'meet the parents (2000)', 'verdict, the (1982)', 'remember the titans (2000)', 'best in show (2000)', 'old yeller (1957)', 'magnolia (1999)', 'sabrina (1995)', 'sleepless in seattle (1993)', 'driving miss daisy (1989)', 'right stuff, the (1983)', 'wonder boys (2000)', "you've got mail (1998)", 'natural, the (1984)', 'pretty woman (1990)', 'frequency (2000)', 'killing fields, the (1984)', 'requiem for a dream (2000)']


##### 8) 회고


- Matrix Factorization을 이용하여 성능보다는 동작 위주의 추천 시스템을 실습해보았다. 
- 전처리는 이전부터 해오던 내용이었지만 실제 동작하는 시스템과 비슷해 재미있었다.  
- DataFrame에 데이터를 넣거나, 조작하는 부분은 아직도 많이 헤매는 부분이다.  
- 큰 매트릭스를 간단히 표현하는 CSR Matrix는 임베디드 환경에서 유용할 것으로 보이나, 이를 이용하여 모델 학습하는데는 많은 시간이 걸렸다.